In [ ]:
# Python Package Imports
import sys
import os 
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
# DSUB set up
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')
%env USER_NAME={USER_NAME}
%env JOB_NAME=subset_and_filter_for_ldsc

In [ ]:
# This and next cell randomly selects AoU IDs to match required population composition

biobank = 'meta_analysis_mixed' # job name
n = 5000 # sample size for LD score calculations. 

# Summix2 Fractions
fractions=[0.7, 0.17, 0.045, 0.082, 0, 0]

counts = [int(round(n*x, 0)) for x in fractions]
pop_fractions = pd.DataFrame({'POP':["eur", "afr", "amr", "eas", "sas", "mid"],
                            'Counts': counts})


# Read AoU major continental population predictions
pop = pd.read_csv("ancestry_preds.tsv", sep = '\t')

mapper = pop_fractions.set_index('POP')['Counts'].to_dict()

IDs = pop.groupby('ancestry_pred').apply(lambda x: x.sample(n=mapper.get(x.name))).reset_index(drop = True)

In [ ]:
%env BIOBANK={biobank}

In [ ]:
ID_list = pd.DataFrame({'FID':IDs['research_id'],
                        'IID':IDs['research_id'],
                        'zero':0})

# Actual google-bucket path is obscure for safety
ID_list.to_csv("gs~cov-ldsc/summix/"+ biobank + ".idlist", 
               sep = '\t',
               index=False, header=False)

In [ ]:
# creating task file for parallelization
PARAMETER_FILENAME = 'gs~cov-ldsc/task_files/subset_and_map_bgen_task.tsv'

df = pd.DataFrame(data={
    '--env CHR': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22],
    '--input BGEN': ['gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr1_filtered_v7.bgen',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr2_filtered_v7.bgen',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr3_filtered_v7.bgen',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr4_filtered_v7.bgen',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr5_filtered_v7.bgen',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr6_filtered_v7.bgen',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr7_filtered_v7.bgen',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr8_filtered_v7.bgen',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr9_filtered_v7.bgen',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr10_filtered_v7.bgen', 
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr11_filtered_v7.bgen',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr12_filtered_v7.bgen',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr13_filtered_v7.bgen',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr14_filtered_v7.bgen',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr15_filtered_v7.bgen',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr16_filtered_v7.bgen',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr17_filtered_v7.bgen',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr18_filtered_v7.bgen',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr19_filtered_v7.bgen',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr20_filtered_v7.bgen',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr21_filtered_v7.bgen',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr22_filtered_v7.bgen'],
    '--input SAMPLE': ['gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr1_filtered_v7.sample',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr2_filtered_v7.sample',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr3_filtered_v7.sample',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr4_filtered_v7.sample',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr5_filtered_v7.sample',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr6_filtered_v7.sample',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr7_filtered_v7.sample',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr8_filtered_v7.sample',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr9_filtered_v7.sample',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr10_filtered_v7.sample', 
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr11_filtered_v7.sample',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr12_filtered_v7.sample',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr13_filtered_v7.sample',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr14_filtered_v7.sample',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr15_filtered_v7.sample',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr16_filtered_v7.sample',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr17_filtered_v7.sample',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr18_filtered_v7.sample',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr19_filtered_v7.sample',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr20_filtered_v7.sample',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr21_filtered_v7.sample',
             'gs~VDS_FINAL_PULL/mac_filtered_BGEN/chr22_filtered_v7.sample']
})

# select chromosomes to run
df=df.iloc[0:22,:]
df.to_csv(PARAMETER_FILENAME, sep='\t', index=False)

In [ ]:
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

In [ ]:
%%writefile ~/subset_and_map_bgen.bash

#!/bin/bash

# Sample file must be in the following format (tab separated) first row is 0\t0\t0 followed by FID\tIID\t0
# '''
# 0       0       0
# 1004866 1004866 0
# 2928323 2928323 0
# '''

set -o errexit
set -o nounset

# extracting directory name
GENETIC_MAP_PATH="$(dirname "${GENETIC_MAP}")"


# plink2 --bgen ${BGEN} ref-first --sample ${SAMPLE} \
#      --make-bed \
#      --out ${output}/test

            
# converting bgen to bed
# subsetting to IDs in $file
plink2 --bgen ${BGEN} ref-first --sample ${SAMPLE} \
    --make-bed \
    --keep ${ID_FILE} \
    --out "chr"
            
# removing high LD regions, multiallelic sites andfiltering by missingness, hwe, maf
plink2 --bfile "chr" \
    --exclude range ${HIGH_LD} \
    --max-alleles 2 \
    --rm-dup force-first \
    --geno 0.1 \
    --hwe 1e-6 \
    --maf 0.05 \
    --make-bed \
    --out "chr_filtered"
  
# # Optional to remove variants with empty names 
# # Our BGENs should not have any variants like this, however
# #          plink2 --bfile "chr"$i"_filtered"
# #              --exclude missing_name_snps \ # remove SNPs with empty IDs
# #              --make-bed \
# #              --out "chr"$i"_filtered"

# Adding genetic map
plink --bfile "chr_filtered" \
    --cm-map ${GENETIC_MAP_PATH}/genetic_map_chr${CHR}_hg38.txt ${CHR} \
    --make-bed \
    --out ${output}/chr${CHR}_map

In [ ]:
# copying script to cloud to be invoked by dsub later
!gsutil cp /home/jupyter/subset_and_map_bgen.bash ${WORKSPACE_BUCKET}/data/cov-ldsc/scripts/

In [ ]:
%%writefile ~/aou_dsub.bash

#!/bin/bash

# This shell function passes reasonable defaults for several dsub parameters, while
# allowing the caller to override any of them. It creates a nice folder structure within
# the workspace bucket for dsub log files.

# --[ Parameters ]--
# any valid dsub parameter flag

#--[ Returns ]--
# the job id of the job created by dsub

#--[ Details ]--
# The first five parameters below should always be those values when running on AoU RWB.

# Feel free to change the values for --user, --regions, --logging, and --image if you like.

# Note that we insert some job data into the logging path.
# https://github.com/DataBiosphere/dsub/blob/main/docs/logging.md#inserting-job-data

function aou_dsub () {

  # Get a shorter username to leave more characters for the job name.
  local DSUB_USER_NAME="$(echo "${OWNER_EMAIL}" | cut -d@ -f1)"

  # For AoU RWB projects network name is "network".
  local AOU_NETWORK=network
  local AOU_SUBNETWORK=subnetwork

  dsub \
      --provider google-cls-v2 \
      --user-project "${GOOGLE_PROJECT}"\
      --project "${GOOGLE_PROJECT}"\
      --image 'marketplace.gcr.io/google/ubuntu1804:latest' \
      --network "${AOU_NETWORK}" \
      --subnetwork "${AOU_SUBNETWORK}" \
      --service-account "$(gcloud config get-value account)" \
      --user "${DSUB_USER_NAME}" \
      --regions us-central1 \
      --logging "${WORKSPACE_BUCKET}/dsub/logs/{job-name}/{user-id}/$(date +'%Y%m%d/%H%M%S')/{job-id}-{task-id}-{task-attempt}.log" \
      "$@"
}

In [ ]:
%%bash

# this is optional and allows using the function in the terminal
echo source ~/aou_dsub.bash >> ~/.bashrc

In [ ]:
%%bash --out JOB_NAME

source ~/aou_dsub.bash

# Get a shorter username to leave more characters for the job name.
DSUB_USER_NAME="$(echo "${OWNER_EMAIL}" | cut -d@ -f1)"
MACHINE_TYPE="n2-standard-4"

# For AoU RWB projects network name is "network".
AOU_NETWORK=network
AOU_SUBNETWORK=subnetwork

aou_dsub \
      --tasks "${PARAMETER_FILENAME}" \
      --machine-type ${MACHINE_TYPE} \
      --image us.gcr.io/broad-dsp-gcr-public/terra-jupyter-aou:2.1.19 \
      --disk-size 512 \
      --boot-disk-size 50 \
      --user "${DSUB_USER_NAME}" \
      --name "${JOB_NAME}" \
      --input ID_FILE="gs~cov-ldsc/summix/${BIOBANK}.idlist" \
      --input GENETIC_MAP='gs~cov-ldsc/genetic_map_hg38/*' \
      --input HIGH_LD='gs~cov-ldsc/high_LD_regions/high-LD-regions-hg38-GRCh38.bed' \
      --input MISSING_NAMES='gs~cov-ldsc/missing_names/missing_name_snps' \
      --output-recursive output="${WORKSPACE_BUCKET}/data/cov-ldsc/output/${BIOBANK}" \
      --script "${WORKSPACE_BUCKET}/data/cov-ldsc/scripts/subset_and_map_bgen.bash"

In [ ]:
# to make JOB_ID available from %%bash cells
%env JOB_ID={JOB_NAME}

In [ ]:
%%bash
# Check status
dstat \
    --provider google-cls-v2 \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*' \